In [5]:
import numpy as np
import math
with open('input.dat', 'r') as file:
    text = file.read()
x = text.split('\n')

num = int(x[0])
tau = float(x[1])
x = x[2:-1]
data = []
for line in x:
    data.append(line.split(' '))
for i in range(10):  # delete unused space caused by split
    data[i] = data[i][1:]
for i in range(num):
    data[i][0] = int(data[i][0])
    data[i][1] = float(data[i][1])
    data[i][2] = int(data[i][2])
#print(num, tau, data)
data = np.copy(data)

In [6]:
def SumWorstResponse(num, tau, data):
    response_sum = 0
    B = [0] * num
    C = [0] * num
    Q = [0] * num
    R = [0] * num
    wrong = False
    for i in range(num):
        #find B_i from same to smaller priority(bigger i)
        for j in range(num):
            if data[j][0] >= data[i][0]:
                if data[j][1] >  B[i]:
                    B[i] = data[j][1]
        Q[i] = B[i]

        # find RHS(right hand side) 
        while True:
            r_sum = 0
            for j in range(num):
                if data[j][0] < data[i][0]:
                    r_sum += math.ceil((Q[i]+tau)/data[j][2])*data[j][1]
            RHS = B[i] + r_sum

            if RHS+data[i][1] > data[i][2]:
                #print('System not schedulable')
                response_sum = 5999
                wrong = True
                break
            if Q[i] == RHS:
                R[i] = Q[i] + data[i][1]
                #print('Worst-case response time of {}: {:.2f}'.format(i, R[i]))
                response_sum += R[i]
                break
            else:
                Q[i] = RHS
        if wrong == True:
            break
    #print("Current sum: ", response_sum)
    return response_sum

In [7]:
def SwapPriority(data):
    new_data = np.copy(data)
    c1, c2 = np.random.randint(17), np.random.randint(17)
    #print("change ", c1, c2)
    new_data[c1][0], new_data[c2][0] = data[c2][0], data[c1][0]
    #print("New data: ", new_data[:,0])
    return new_data

In [8]:
# initial
T = 1999
r = 0.995
cost_S = SumWorstResponse(num, tau, data)
# S*
S_star = data
cost_S_star = cost_S

while T > 1:
    # random swap 2 priority
    S_pron = SwapPriority(data)
    # S'
    cost_S_pron = SumWorstResponse(num, tau, S_pron)
    delta_C = cost_S_pron - cost_S
    if cost_S_pron < cost_S_star:
        S_star = S_pron
        cost_S_star = cost_S_pron
    if delta_C <= 0:
        data = S_pron
    else:
        p = math.exp(-delta_C/T)
        #print(p)
        if np.random.rand() < p:
            #print("Cost bigger but swap")
            data = S_pron
    #print(data[:,0])
    #print("Best sum: ", cost_S_star)
    T = r*T
for k in range(17):
    print(int(S_star[k][0]))
print("Best sum: ", cost_S_star)

14
6
0
4
1
3
9
2
7
5
12
16
13
10
15
11
8
Best sum:  204.75999999999996
